In [ ]:
%pylab inline

## Dirac Comb

http://en.wikipedia.org/wiki/Dirac_delta_function

http://en.wikipedia.org/wiki/Dirac_comb

An impulse train (Dirac comb) can be expressed as an infinite sum of harmonics of the same amplitude, whose fundamental is the frequency of the impulse train. 

In [ ]:
linspace(0, 1, 10) # inclusive

$$ x(t) = cos(\omega t)$$ 

In [ ]:
wt = linspace(0, 6 * pi, 20000)

In [ ]:
oscillation = cos(wt)
plot(oscillation);

How do I get the next harmonic i.e. double the frequency of this sinusoid?

$ x(t) = cos(2 \pi f t)$

In [ ]:
f = 3
w = 2 * pi * f

In [ ]:
phase_t = linspace(0, w, 50000)
impulse = cos(phase_t) + cos(2*phase_t)
impulse /= 2
plot(impulse);

In [ ]:
impulse = cos(phase_t) + cos(2*phase_t) + cos(3*phase_t) + cos(4*phase_t)
impulse /= 4
plot(impulse);

equally weighted harmonics form a impulse

In [ ]:
N = 100
phase = linspace(0, 6*pi, 50000)
harmonics = arange(N) + 1
impulse = zeros_like(phase) # zeros(phase.shape)
for harmonic in harmonics:
    impulse += cos(harmonic * phase)
    
impulse /= N
plot(impulse);

[Gibbs phenomenon](https://en.wikipedia.org/wiki/Gibbs_phenomenon)

In [ ]:
plot(impulse)
xlim((0, 5000))

Why do we use cosine? Isn't sine the same thing?

In [ ]:
N = 100
phase = linspace(0, 6*pi, 50000)
harmonics = arange(N) + 1
impulse = zeros_like(phase)
for harmonic in harmonics:
    impulse += sin(harmonic * phase)
    
impulse /= N
plot(impulse);

(See [Even and odd functions](https://en.wikipedia.org/wiki/Even_and_odd_functions) - optional)

## Sampling

Sampling can be described as a multiplication between the Dirac comb and the signal:

In [ ]:
comb = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] * 25
comb [-1] = 1 # unholy trick to make some things simpler later...
# plot(comb)
stem(comb) # a different kind of plot
ylim(0, 1.1)
xticks(())
title("Dirac Comb")

To be strict, this that I just created is the Kronecker delta because it's discrete, but let's assume it's continuous

https://en.wikipedia.org/wiki/Kronecker_delta

Now let's get a function that we can sample:

In [ ]:
from scipy.special import jn
jn(0, 0), jn(0, 1)

Bessel functions: http://en.wikipedia.org/wiki/Bessel_function

In [ ]:
jn(1,0), jn(7,0.3), jn(1, 1)

Our "Continuous" function: 

In [ ]:
x = linspace(0,10, 500)
plot(x, jn(1,x))
title('Bessel function of the first type, order 1');

In [ ]:
x = linspace(0,10, 500)
plot(x, jn(1,x))
plot(x, jn(1,x) * comb); # plot again in the same cell for superposition

In [ ]:
plot(x, jn(1,x) * comb);

Now just keep the values when the Dirac comb == 1

In [ ]:
sampled = jn(1,x) * comb
samples = []
for i in range(len(comb)):
    if comb[i] == 1:
        samples.append(sampled[i])

plot(samples, 'o');

In [ ]:
len(samples)

In [ ]:
x_sampled = linspace(0,10, len(samples))
plot(x_sampled, samples)
x = linspace(0,10, 500)
plot(x, jn(1,x));

connect our samples with lines

In [ ]:
plot(x_sampled, samples, 'x-')
plot(x, jn(1,x));
xlim((0.5, 3))
ylim((0.3, 0.65))

the original signal compared to the piecewise linear reconstruction

In [ ]:
# upsample using linear interpolation
from scipy.interpolate import interp1d
interpf = interp1d(linspace(0,10, len(samples)), samples)
sqe = (jn(1,x) - interpf(x))**2
plot(interpf(x))
plot(jn(1,x))

twinx()
plot(sqe, 'r')
axis(ymax= sqe.max())
ylabel('squared error', color='r',fontsize=18);

Hot tip: Shift-Tab on functions for documentation

In [ ]:
sqe = (jn(1,x) - interpf(x))**2
plot(interpf(x))
plot(jn(1,x))
ylim((0.3, 0.65))

twinx()
axis(ymax= sqe.max())
gca().set_ylabel('squared error', color='r',fontsize=18)
plot(sqe, 'r')
xlim((50,150));

...now showing the "error"---how well does the reconstruction do?

In [ ]:
MSE = sqe.mean()
print(MSE)


Mean squared error is a common way to quantify the difference between two signals:

$$MSE = \frac{1}{n}\sum_{i=1}^n(X_1 - X_2)^2$$

***Note:* This does not mean that the digitized signal has that error!**

It just means that if we reconstruct the signal by drawing straight lines we get this error.

# Sampling theorem

If a function $f(x)$  contains no frequencies higher than B hertz, it is completely determined by its ordinates (value on the y axis) at a series of points spaced 1/(2B) seconds apart. Or.. you must sample a signal at a rate two times that of it's highest frequency if you hope to capture the signal perfectly.

In [ ]:
# the recorded (aka sampled) signal which is 
# sampled at 10 Hz (10B Hz)
x = linspace(0, 2 * pi, 10)
plot(x, sin(x))

# the original signal which has frequency B = 1
x0 = linspace(0, 2 * pi, 100)
plot(x0, sin(x0))

But.... isn't there loss here? Even though the samples are spaced closer than 1/B???



.

.


.


.

.

.


.


.

Although there is a difference between the different sampled signals, no information from the original signal has been lost!

i.e. when we do the ADC there will be no difference between either. (in theory...  it's another thing in practice!)

## Foldover/Aliasing

But there will be a difference if there are less than two points per sine oscillation, i.e. when the frequency we are sampling is higher that sr/2 (Nyquist frequency).

In [ ]:
phs = linspace(0, 10 * 2 * pi, 300)
plot(sin(phs))

Anything less than 20 points will cause problems:

In [ ]:
phs = linspace(0, 10 * 2 * pi, 300)
plot(phs, sin(phs))
phs = linspace(0, 10 * 2 * pi, 12)
plot(phs, sin(phs), 'o--')

Here our ADC finds a single cycle in the 10 cycles of the original signal. This is bad data.

In [ ]:
from ipywidgets import interact
dense = linspace(0, 10 * 2 * pi, 300, endpoint=False)

def plotsin(steps):
    plot(dense, cos(dense))
    phs = linspace(0, 10 * 2 * pi, steps, endpoint=False)
    plot(phs, cos(phs), 'o--')
    show()

interact(plotsin, steps=(1, 30))
pass

In [ ]:
phs = linspace(0, 10 * 2 * pi, 300)
plot(phs, sin(phs))
phs = linspace(0, 10 * 2 * pi, 21)
plot(phs, sin(phs), 'o--')

In discrete sampling twice the Nyquist frequency is the same as DC (i.e. frequency 0)...

In [ ]:
phs = linspace(0, 10 * 2 * pi, 300)
plot(phs, cos(phs))
phs = linspace(0, 10 * 2 * pi, 11)
plot(phs, cos(phs), 'o--')

The frequency of the foldover component is:

$$ f_{ALIAS} = \frac{f_{sr}}{2} - (f_0 - \frac{f_{sr}}{2})$$

i.e. fold/mirror the frequency around the Nyquist frequency.

$$ f_{ALIAS} = f_{sr} - f_0$$

More strictly:

$$ f_{ALIAS} = f_{sr} - (f_0\pmod {f_{sr}})$$

The frequency wraps around the sampling frequency.


## Quantization

Once the signal has been sampled, a value needs to be assigned to it.

In [ ]:
x = linspace(0, 2*pi, 300)
f = sin(x)
plot(f);

In [ ]:
f = sin(3 * x)
f2 = (f*2).astype(int)
plot(f) 
plot(f2); 

These 3 different values can be encoded in 2 bits.

In [ ]:
f = sin(x)
f2 = (f*2).astype(int)
f4 = (f*4).astype(int)
plot(f)
plot(f2)
plot(f4/3.0)
legend(['sine', '2-bit', '3-bit']);

In [ ]:
2**2, 2**3

In [ ]:
2**16

In [ ]:
#integer representations
x = linspace(0, 2*pi, 100000)
f = sin(x)

def dothings(BITS):
    N = BITS # number of bits
    max_value = 2**(N-1) - 1
    fN = (f*(max_value)).astype(int16)
    plot(sin(x)*max_value)
    plot(fN);
    show()
    
interact(dothings, BITS=(3, 8))
pass

In [ ]:
x = linspace(0, 2*pi, 100000)
f = sin(x)
N = 16 # number of bits
max_value = 2**(N-1) - 1
f16 = (f*(max_value)).astype(int16)
plot(f16, 'x-' )
xlim((0, 50))
ylim((0, 80))

In [ ]:
N = 5 # number of bits
max_value = 2**(N-1) - 1
f8 = (f*(max_value)).astype(int8)
plot(f8)

In [ ]:
plot(f8, 'o')
xlim((0,20))
ylim((0, 10))
grid();

In [ ]:
2**24

## Dynamic range
(See [Dynamic range](https://en.wikipedia.org/wiki/Dynamic_range))
also, [contrast ratio](https://en.wikipedia.org/wiki/Contrast_ratio)

In [ ]:
N = 16
20 * log10((2 ** (N - 1))/1)

def dynrangedb(N):
    return 20 * log10((2 ** (N - 1))/1)

In [ ]:
print(dynrangedb(8), dynrangedb(16), dynrangedb(24))

In [ ]:
20*log10(0.5)

In [ ]:
20*log10(60/30)

Half the linear amplitude scale is only 6 dB!

## Amplitude encoding

In [ ]:
22050 * 16

In [ ]:
352800 * 60

In [ ]:
21168000/(8 * 1024)

When the values for amplitude are stored directly from the linear measurements of energy, this form of "encoding" is known as LPCM (Linear Pulse Code Modulation)

http://en.wikipedia.org/wiki/Pulse-code_modulation

Differential Pulse Code modulation stores the difference between samples

In [ ]:
from scipy.io import wavfile

sr,audio = wavfile.read('media/passport.wav')
plot(audio)
print(audio.max(), audio.min(), sr)

In [ ]:
2**16

In [ ]:
audio.dtype

In [ ]:
x = linspace(0, 2*pi, 50)
plot(sin(x), 'o')
plot(diff(sin(x)))

In [ ]:
dpcm = diff(audio)
plot(dpcm)
dpcm.max(), dpcm.min()

In [ ]:
log(max(dpcm.max(), abs(dpcm.min())))/log(2)

In [ ]:
log(max(audio.max(), abs(audio.min())))/log(2)

The differential encoding would save 1 bit

ADPCM (Adaptive DPCM) uses different resolutions depending on what it needs.

Delta modulation encodes using only 1 bit to describe the change, and so requires a higher sampling rate. e.g. DSD

http://en.wikipedia.org/wiki/Direct_Stream_Digital

By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)